<a href="https://colab.research.google.com/github/juanpriolo1992/c2021/blob/gh-pages/Grupo5_COVID_Viz_Python_JuanPriolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import os


In [ ]:
!wget -c https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.zip
!unzip Covid19Casos.zip

--2021-08-01 23:01:52--  https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.zip
Resolving sisa.msal.gov.ar (sisa.msal.gov.ar)... 186.33.232.246
Connecting to sisa.msal.gov.ar (sisa.msal.gov.ar)|186.33.232.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235818498 (225M) [application/zip]
Saving to: ‘Covid19Casos.zip’

Covid19Casos.zip    100%[===================>] 224.89M  12.9MB/s    in 27s     

2021-08-01 23:02:21 (8.23 MB/s) - ‘Covid19Casos.zip’ saved [235818498/235818498]

Archive:  Covid19Casos.zip
  inflating: Covid19Casos.csv        


In [ ]:
dates_to_parse = ['fecha_inicio_sintomas','fecha_diagnostico']#,'fecha_apertura','fecha_internacion','fecha_cui_intensivo','fecha_fallecimiento','fecha_diagnostico','ultima_actualizacion']
dtypes = {'carga_provincia_id' : object, 
          'residencia_provincia_id' : object, 
          'residencia_departamento_id' : object}
df = pd.read_csv("Covid19Casos.csv", dtype= dtypes,parse_dates=dates_to_parse)

In [ ]:
df.dtypes

id_evento_caso                               int64
sexo                                        object
edad                                       float64
edad_años_meses                             object
residencia_pais_nombre                      object
residencia_provincia_nombre                 object
residencia_departamento_nombre              object
carga_provincia_nombre                      object
fecha_inicio_sintomas               datetime64[ns]
fecha_apertura                              object
sepi_apertura                                int64
fecha_internacion                           object
cuidado_intensivo                           object
fecha_cui_intensivo                         object
fallecido                                   object
fecha_fallecimiento                         object
asistencia_respiratoria_mecanica            object
carga_provincia_id                          object
origen_financiamiento                       object
clasificacion                  

In [ ]:
df.head(1000)

,id_evento_caso,sexo,edad,edad_años_meses,residencia_pais_nombre,residencia_provincia_nombre,residencia_departamento_nombre,carga_provincia_nombre,fecha_inicio_sintomas,fecha_apertura,sepi_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,asistencia_respiratoria_mecanica,carga_provincia_id,origen_financiamiento,clasificacion,clasificacion_resumen,residencia_provincia_id,fecha_diagnostico,residencia_departamento_id,ultima_actualizacion
0,1000000,M,54.0,Años,Argentina,CABA,SIN ESPECIFICAR,Buenos Aires,NaT,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,06,Privado,Caso Descartado,Descartado,02,2020-06-09,0,2021-07-31
1,10000000,F,52.0,Años,Argentina,CABA,SIN ESPECIFICAR,CABA,NaT,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,02,Público,Caso Descartado,Descartado,02,2021-04-08,0,2021-07-31
2,10000001,F,22.0,Años,Argentina,Buenos Aires,Lomas de Zamora,Buenos Aires,2021-04-05,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,06,Público,Caso Descartado,Descartado,06,2021-04-08,490,2021-07-31
3,10000002,F,44.0,Años,Argentina,Buenos Aires,La Matanza,CABA,NaT,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,02,Público,Caso Descartado,Descartado,06,2021-04-06,427,2021-07-31
4,10000003,F,34.0,Años,Argentina,CABA,COMUNA 01,CABA,NaT,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,02,Público,Caso Descartado,Descartado,02,2021-04-08,001,2021-07-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10000939,F,45.0,Años,SIN ESPECIFICAR,CABA,SIN ESPECIFICAR,CABA,NaT,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,02,Público,Caso Descartado,Descartado,02,2021-04-06,0,2021-07-31
996,1000094,F,55.0,Años,Argentina,CABA,SIN ESPECIFICAR,CABA,NaT,2020-06-01,23,NaN,NO,NaN,NO,NaN,NO,02,Privado,Caso Descartado,Descartado,02,2020-06-01,0,2021-07-31
997,10000940,F,59.0,Años,Argentina,Córdoba,Capital,Córdoba,NaT,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,14,Público,Caso Descartado,Descartado,14,2021-04-07,014,2021-07-31
998,10000941,F,9.0,Años,Argentina,Buenos Aires,Trenque Lauquen,Buenos Aires,NaT,2021-04-08,14,NaN,NO,NaN,NO,NaN,NO,06,Público,Caso Descartado,Descartado,06,2021-04-11,826,2021-07-31


# **Personas fallecidas por edad con línea en valor máximo**


In [ ]:
df_fallecido_edad = df[(df['fallecido'] == 'SI') &
                   (df['edad'] >= 0) & (df['edad'] < 120 ) ]
df_fallecido_edad1= df_fallecido_edad.groupby(['edad']).size().to_frame(name="cantidad de muertos").reset_index()


In [ ]:
df_fallecido_edad1

,edad,cantidad de muertos
0,0.0,1
1,1.0,84
2,2.0,45
3,3.0,22
4,4.0,23
...,...,...
106,106.0,7
107,107.0,5
108,108.0,1
109,109.0,1


In [ ]:
source = df_fallecido_edad1

grafico = alt.Chart(source, title="Fallecidos por edad").mark_bar().encode(
    x='edad:Q',
    y="cantidad de muertos:Q",
    tooltip = 'cantidad de muertos:Q',
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.edad == 77,  # If the year is 1810 this test returns True,
        alt.value('red'),     # which sets the bar orange.
        alt.value('grey'),   # And if it's not true it sets the bar steelblue.
        
    )
).properties(width=600)
grafico.save("prueba.html")

#**Personas fallecidas por edad y por sexo**

---

In [ ]:
df_fallecido_edad = df[(df['fallecido'] == 'SI') &
                   (df['edad'] >= 0) & (df['edad'] < 120 ) & (df['sexo'] != 'NR') ]
df_fallecido_edad1= df_fallecido_edad.groupby(['edad','sexo']).size().to_frame(name="cantidad de muertos").reset_index()


In [ ]:
df_fallecido_edad1.head(10)

,edad,sexo,cantidad de muertos
0,0.0,F,1
1,1.0,F,19
2,1.0,M,19
3,2.0,F,20
4,2.0,M,23
5,3.0,F,12
6,3.0,M,9
7,4.0,F,9
8,4.0,M,14
9,5.0,F,5


In [ ]:
df_fallecido_edad = df[(df['fallecido'] == 'SI') &
                   (df['edad'] >= 0) & (df['edad'] < 120 ) & (df['sexo'] != 'NR') ]
df_fallecido_edad2= df_fallecido_edad.groupby(['edad','sexo']).size().to_frame(name="cantidad de muertos").reset_index()

In [ ]:
source = df_fallecido_edad2

grafico2 = alt.Chart(source,title="Fallecidos por edad y sexo").mark_line().encode(
    x=alt.X('edad',axis=alt.Axis( title='Edad')),
    y=alt.Y('cantidad de muertos',axis=alt.Axis( title='Cantidad de fallecidos')),
    color=alt.Color('sexo', legend=alt.Legend(orient="right", title= "Sexo")),
    strokeDash='sexo',
    tooltip = 'cantidad de muertos:Q'
)
grafico2.save("FallecidosXEdadySexo.html")

In [ ]:
source = df_fallecido_edad2

alt.Chart(source,title="Fallecidos por edad y sexo" ).mark_area(opacity=0.25).encode(
    x="edad:Q",
    y=alt.Y("cantidad de muertos:Q", stack=None),
    color="sexo:N",
    tooltip = 'cantidad de muertos:Q'
)

alt.Chart(...)

# **Gráfico para ver proporción de gente que recibió cuidado intensivo del Estado o del sector Privado**

---


In [ ]:
df_intervención_publicavsprivada_provincia = df[(df['cuidado_intensivo'] == 'SI') &
                   (df['edad'] >= 0) & (df['edad'] < 120 ) & (df['sexo'] != 'NR') & (df['residencia_provincia_nombre'] != 'SIN ESPECIFICAR') ]    

df_intervención_publicavsprivada_provincia1= df_intervención_publicavsprivada_provincia.groupby(['residencia_provincia_nombre','origen_financiamiento']).size().to_frame(name="cuidado_intensivo").reset_index()

In [ ]:
df_intervención_publicavsprivada_provincia1.head(23)

,residencia_provincia_nombre,origen_financiamiento,cuidado_intensivo
0,Buenos Aires,Privado,14735
1,Buenos Aires,Público,13184
2,CABA,Privado,5586
3,CABA,Público,3609
4,Catamarca,Privado,51
5,Catamarca,Público,484
6,Chaco,Privado,371
7,Chaco,Público,2453
8,Chubut,Privado,73
9,Chubut,Público,528


In [ ]:
source = df_intervención_publicavsprivada_provincia1

alt.Chart(source, title="Financiamiento Público vs Privado Total" ).mark_bar().encode(
    x='cuidado_intensivo',
    y='origen_financiamiento',
    color=alt.Color('origen_financiamiento'),
       order=alt.Order(
      # Sort the segments of the bars by this field
      'origen_financiamiento',
      sort='descending'
    )
)

alt.Chart(...)

In [ ]:
source = df_intervención_publicavsprivada_provincia1

grafico3 = alt.Chart(source, title="Financiamiento Público vs Privado" ).mark_bar().encode(
    x=alt.X('cuidado_intensivo',axis=alt.Axis( title='Cantidad de ingresos a cuidados intensivos')),
    y=alt.Y('residencia_provincia_nombre',axis=alt.Axis( title='Provincia'),sort = '-x'),
    color=alt.Color('origen_financiamiento',legend=alt.Legend(orient="right", title= "Origen del financiamento")),
       order=alt.Order(
      # Sort the segments of the bars by this field
      'cuidado_intensivo:Q',
      sort='descending'
    ),
    tooltip = 'cuidado_intensivo:Q'
)
grafico3.save("OrigenDelFinanciamento.html")

In [ ]:
source = df_intervención_publicavsprivada_provincia1

alt.Chart(source).mark_trail().encode(
    x='year:T',
    y='wheat:Q',
    size='wheat:Q'
)

In [ ]:
dates_to_parse = ['fecha_internacion']#,'fecha_apertura','fecha_internacion','fecha_cui_intensivo','fecha_fallecimiento','fecha_diagnostico','ultima_actualizacion']
dtypes = {'carga_provincia_id' : object, 
          'residencia_provincia_id' : object, 
          'residencia_departamento_id' : object}
df = pd.read_csv("Covid19Casos.csv", dtype= dtypes,parse_dates=dates_to_parse)

In [ ]:
df_confirmado_edad = df[(df['cuidado_intensivo'] == 'SI') &
                   (df['edad'] >= 0) & (df['edad'] < 120 ) & (df['edad_años_meses'] == 'Años')]
df_fechasOK = df_confirmado_edad[df_confirmado_edad['fecha_diagnostico'] > '2020-01-01']
df_agrupado_financiamiento = df_fechasOK.groupby(['fecha_diagnostico','origen_financiamiento'])['edad'].count().to_frame(name = "cantidad").reset_index()
#df_agrupado_provinciaxdia.columns = ['Fecha','Provincia','CasosRegistrados']
#df_agrupado_financiamiento['AcumuladoCasos'] = df_agrupado_financiamiento.groupby(['origen_financiamiento']).cumsum()
#df_agrupado_provinciaxdia = df_agrupado_provinciaxdia[df_agrupado_provinciaxdia['Provincia'] != 'SIN ESPECIFICAR']

In [ ]:
df_agrupado_financiamiento

In [ ]:
source = df_agrupado_financiamiento

alt.Chart(source, title = 'Origen Financiamiento').mark_line().encode(
    x='fecha_diagnostico:T',
    y='cantidad',
    color='origen_financiamiento',
    tooltip='cantidad:N',

)

alt.Chart(...)

In [ ]:
source = df_agrupado_financiamiento

alt.Chart(source, width=450, height=250, title="Vacunas utilizadas en Argentina").mark_area(
    interpolate='basis',
    line=False
).encode(
    x=alt.X('fecha_diagnostico:T', title="fecha"),
    # y=alt.Y('sum(cantidad)', stack="normalize"),
    y=alt.Y('sum(cantidad)', stack='center', axis=None),
    color='origen_financiamiento:N',
    tooltip='cantidad:N'
)

alt.Chart(...)